In [1]:
import jax.random as jr
import jax.numpy as jnp
import flax.linen as nn

from jaxtyping import Float, Array
from modules import LayerNorm, Embed, PosEmbed, TransformerBlock, Unembed


In [2]:
class Transformer(nn.Module):
    model_dim: int = 768
    layer_norm_eps: Float = 1e-5
    vocab_dim: int = 50_257
    context_length: int = 1024
    num_heads: int = 12
    head_dim: int = 64
    mlp_dim: int = 3072
    num_layers: int = 12

    @nn.compact
    def __call__(self, tokens) -> Array:
        embedding = Embed(
            features=self.model_dim,
            num_embeddings=self.vocab_dim,
        )(tokens)

        pos_embedding = PosEmbed(
            features=self.model_dim,
            context_length=self.context_length,
        )(tokens)

        x = embedding + pos_embedding

        for _ in range(self.num_layers):
            x = TransformerBlock(
                num_heads=self.num_heads,
                head_dim=self.head_dim,
                model_dim=self.model_dim,
                mlp_dim=self.mlp_dim,
                epsilon=self.layer_norm_eps,
            )(x)

        x = LayerNorm(epsilon=self.layer_norm_eps)(x)

        logits = Unembed(
            features=self.model_dim,
            num_embeddings=self.vocab_dim,
        )(x)

        return logits


print(Transformer().tabulate(jr.PRNGKey(0), jnp.ones((1, 1024), dtype=jnp.int32)))


                              Transformer Summary                               
┏━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ path          ┃ module        ┃ inputs        ┃ outputs      ┃ params        ┃
┡━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│               │ Transformer   │ int32[1,1024] │ float32[1,1… │               │
├───────────────┼───────────────┼───────────────┼──────────────┼───────────────┤
│ Embed_0       │ Embed         │ int32[1,1024] │ float32[1,1… │ embedding:    │
│               │               │               │              │ float32[5025… │
│               │               │               │              │               │
│               │               │               │              │ 38,597,376    │
│               │               │               │              │ (154.4 MB)    │
├───────────────┼───────────────┼───────────────┼──────────────┼───────────────┤
│ PosEmbed_0    │ PosEmbed 

In [1]:
import datasets

In [2]:
ds = datasets.load_dataset("NeelNanda/pile-10k", split="train").remove_columns("meta")
ds

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset({
    features: ['text'],
    num_rows: 10000
})

In [4]:
print(ds[0]['text'][:100])

It is done, and submitted. You can play “Survival of the Tastiest” on Android, and on the web. Playi
